In [ ]:
try:
    import google.colab
    from google.colab import drive
    drive.mount('/content/drive', force_remount = True)
    IN_COLAB = True
except:
    IN_COLAB = False

Mounted at /content/drive


In [ ]:
if IN_COLAB:
  !pip install transformers
  !pip install datasets
  !pip install evaluate
  !pip install sentencepiece

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 7.1/7.1 MB 110.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 224.5/224.5 kB 24.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 7.8/7.8 MB 121.4 MB/s eta 0:00:00
Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 474.6/474.6 kB 1.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 110.5/110.5 kB 14.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 212.5/212.5 kB 23.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 134.3/134.3 kB 16.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.0/1.0 MB 63.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 114.5/114.5 kB 12.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 268.8/268.8 kB 29.6 MB/s eta 0:00

In [ ]:
import os
import torch

import warnings
warnings.filterwarnings('ignore')

if IN_COLAB:
    root_path = '/content/drive/MyDrive/Knowledge/MSIT/Research/Completed Work/InstructABSA'
else:
    root_path = 'Enter local path'
    
use_mps = True if torch.has_mps else False
os.chdir(root_path)

import warnings
warnings.filterwarnings('ignore')
import pandas as pd

from InstructABSA.data_prep import DatasetLoader
from InstructABSA.utils import T5Generator, T5Classifier
from instructions import InstructionsHandler
from transformers import AutoTokenizer, AutoModelForSeq2SeqLM

# Aspect Term Extraction

## InstructABSA - 1 - Laptops 2014

In [ ]:
task_name = 'ate'
model_out_path = 'kevinscaria/ate_tk-instruct-base-def-pos-laptops'
print('Model output path: ', model_out_path)

# Load the data
id_train_file_path = './Dataset/SemEval14/Train/Laptops_Train.csv'
id_test_file_path = './Dataset/SemEval14/Test/Laptops_Test.csv'
id_tr_df = pd.read_csv(id_train_file_path)
id_te_df = pd.read_csv(id_test_file_path)

ood_train_file_path = './Dataset/SemEval14/Train/Restaurants_Train.csv'
ood_test_file_path = './Dataset/SemEval14/Test/Restaurants_Test.csv'
ood_tr_df = pd.read_csv(ood_train_file_path)
ood_te_df = pd.read_csv(ood_test_file_path)

# Get the input text into the required format using Instructions
instruct_handler = InstructionsHandler()

# Set instruction_set1 for InstructABSA-1 and instruction_set2 for InstructABSA-2
instruct_handler.load_instruction_set1()

# Set bos_instruct1 for lapt14 and bos_instruct2 for rest14. For other datasets, modify the insructions.py file.
loader = DatasetLoader(id_tr_df, id_te_df, ood_tr_df, ood_te_df)
if loader.train_df_id is not None:
    loader.train_df_id = loader.create_data_in_ate_format(loader.train_df_id, 'term', 'raw_text', 'aspectTerms', instruct_handler.ate['bos_instruct1'], instruct_handler.ate['eos_instruct'])
if loader.test_df_id is not None:
    loader.test_df_id = loader.create_data_in_ate_format(loader.test_df_id, 'term', 'raw_text', 'aspectTerms', instruct_handler.ate['bos_instruct1'], instruct_handler.ate['eos_instruct'])

if loader.train_df_ood is not None:
    loader.train_df_ood = loader.create_data_in_ate_format(loader.train_df_ood, 'term', 'raw_text', 'aspectTerms', instruct_handler.ate['bos_instruct2'], instruct_handler.ate['eos_instruct'])
if loader.test_df_ood is not None:
    loader.test_df_ood = loader.create_data_in_ate_format(loader.test_df_ood, 'term', 'raw_text', 'aspectTerms', instruct_handler.ate['bos_instruct2'], instruct_handler.ate['eos_instruct'])


# Model inference - Loading from Checkpoint
t5_exp = T5Generator(model_out_path)

# Tokenize Datasets
id_ds, id_tokenized_ds, ood_ds, ood_tokenized_ds = loader.set_data_for_training_semeval(t5_exp.tokenize_function_inputs)

# Get prediction labels - In Domain Training set   
id_tr_pred_labels = t5_exp.get_labels(tokenized_dataset = id_tokenized_ds, sample_set = 'train', batch_size = 16)
id_tr_labels = [i.strip() for i in id_ds['train']['labels']]

# Get prediction labels - In Domain Testing set
id_te_pred_labels = t5_exp.get_labels(tokenized_dataset = id_tokenized_ds, sample_set = 'test', batch_size = 16)
id_te_labels = [i.strip() for i in id_ds['test']['labels']]

# Get prediction labels - Cross Domain Training set   
ood_tr_pred_labels = t5_exp.get_labels(tokenized_dataset = ood_tokenized_ds, sample_set = 'train', batch_size = 16)
ood_tr_labels = [i.strip() for i in ood_ds['train']['labels']]

# Get prediction labels - Cross Domain Testing set
ood_te_pred_labels = t5_exp.get_labels(tokenized_dataset = ood_tokenized_ds, sample_set = 'test', batch_size = 16)
ood_te_labels = [i.strip() for i in ood_ds['test']['labels']]

p, r, f1, _ = t5_exp.get_metrics(id_tr_labels, id_tr_pred_labels)
print('\n** In Domain Metrics **')
print('Train Precision: ', p)
print('Train Recall: ', r)
print('Train F1: ', f1)

p, r, f1, _ = t5_exp.get_metrics(id_te_labels, id_te_pred_labels)
print('Test Precision: ', p)
print('Test Recall: ', r)
print('Test F1: ', f1)

p, r, f1, _ = t5_exp.get_metrics(ood_tr_labels, ood_tr_pred_labels)
print('\n** Cross Domain Metrics **')
print('Train Precision: ', p)
print('Train Recall: ', r)
print('Train F1: ', f1)

p, r, f1, _ = t5_exp.get_metrics(ood_te_labels, ood_te_pred_labels)
print('Test Precision: ', p)
print('Test Recall: ', r)
print('Test F1: ', f1)

Model output path:  kevinscaria/ate_tk-instruct-base-def-pos-laptops


Map:   0%|          | 0/3045 [00:00<?, ? examples/s]

Map:   0%|          | 0/800 [00:00<?, ? examples/s]

Map:   0%|          | 0/3041 [00:00<?, ? examples/s]

Map:   0%|          | 0/800 [00:00<?, ? examples/s]

...

Model loaded to:  cuda


100%|██████████| 191/191 [00:35<00:00,  5.45it/s]


Model loaded to:  cuda


100%|██████████| 50/50 [00:09<00:00,  5.21it/s]


** In Domain Metrics **
Train Precision:  0.9287531806615776
Train Recall:  0.9323116219667944
Train F1:  0.9305289993626514
Test Precision:  0.9326829268292683
Test Recall:  0.9263565891472868
Test F1:  0.9295089936801166

** Cross Domain Metrics **
Train Precision:  0.7404266958424508
Train Recall:  0.5738817044731821
Train F1:  0.6466021736534098
Test Precision:  0.7482517482517482
Test Recall:  0.5627347858752817
Test F1:  0.6423670668953687


## InstructABSA - 2 - Laptops 2014

In [ ]:
task_name = 'ate'
model_out_path = 'kevinscaria/ate_tk-instruct-base-def-pos-neg-neut-laptops'
print('Model output path: ', model_out_path)

# Load the data
id_train_file_path = './Dataset/SemEval14/Train/Laptops_Train.csv'
id_test_file_path = './Dataset/SemEval14/Test/Laptops_Test.csv'
id_tr_df = pd.read_csv(id_train_file_path)
id_te_df = pd.read_csv(id_test_file_path)

ood_train_file_path = './Dataset/SemEval14/Train/Restaurants_Train.csv'
ood_test_file_path = './Dataset/SemEval14/Test/Restaurants_Test.csv'
ood_tr_df = pd.read_csv(ood_train_file_path)
ood_te_df = pd.read_csv(ood_test_file_path)

# Get the input text into the required format using Instructions
instruct_handler = InstructionsHandler()

# Set instruction_set1 for InstructABSA-1 and instruction_set2 for InstructABSA-2
instruct_handler.load_instruction_set2()

# Set bos_instruct1 for lapt14 and bos_instruct2 for rest14. For other datasets, modify the insructions.py file.
loader = DatasetLoader(id_tr_df, id_te_df, ood_tr_df, ood_te_df)
if loader.train_df_id is not None:
    loader.train_df_id = loader.create_data_in_ate_format(loader.train_df_id, 'term', 'raw_text', 'aspectTerms', instruct_handler.ate['bos_instruct1'], instruct_handler.ate['eos_instruct'])
if loader.test_df_id is not None:
    loader.test_df_id = loader.create_data_in_ate_format(loader.test_df_id, 'term', 'raw_text', 'aspectTerms', instruct_handler.ate['bos_instruct1'], instruct_handler.ate['eos_instruct'])

if loader.train_df_ood is not None:
    loader.train_df_ood = loader.create_data_in_ate_format(loader.train_df_ood, 'term', 'raw_text', 'aspectTerms', instruct_handler.ate['bos_instruct2'], instruct_handler.ate['eos_instruct'])
if loader.test_df_ood is not None:
    loader.test_df_ood = loader.create_data_in_ate_format(loader.test_df_ood, 'term', 'raw_text', 'aspectTerms', instruct_handler.ate['bos_instruct2'], instruct_handler.ate['eos_instruct'])

# Model inference - Loading from Checkpoint
t5_exp = T5Generator(model_out_path)

# Tokenize Datasets
id_ds, id_tokenized_ds, ood_ds, ood_tokenized_ds = loader.set_data_for_training_semeval(t5_exp.tokenize_function_inputs)

# Get prediction labels - In Domain Training set   
id_tr_pred_labels = t5_exp.get_labels(tokenized_dataset = id_tokenized_ds, sample_set = 'train', batch_size = 16)
id_tr_labels = [i.strip() for i in id_ds['train']['labels']]

# Get prediction labels - In Domain Testing set
id_te_pred_labels = t5_exp.get_labels(tokenized_dataset = id_tokenized_ds, sample_set = 'test', batch_size = 16)
id_te_labels = [i.strip() for i in id_ds['test']['labels']]

# Get prediction labels - Cross Domain Training set   
ood_tr_pred_labels = t5_exp.get_labels(tokenized_dataset = ood_tokenized_ds, sample_set = 'train', batch_size = 16)
ood_tr_labels = [i.strip() for i in ood_ds['train']['labels']]

# Get prediction labels - Cross Domain Testing set
ood_te_pred_labels = t5_exp.get_labels(tokenized_dataset = ood_tokenized_ds, sample_set = 'test', batch_size = 16)
ood_te_labels = [i.strip() for i in ood_ds['test']['labels']]

p, r, f1, _ = t5_exp.get_metrics(id_tr_labels, id_tr_pred_labels)
print('\n** In Domain Metrics **')
print('Train Precision: ', p)
print('Train Recall: ', r)
print('Train F1: ', f1)

p, r, f1, _ = t5_exp.get_metrics(id_te_labels, id_te_pred_labels)
print('Test Precision: ', p)
print('Test Recall: ', r)
print('Test F1: ', f1)

p, r, f1, _ = t5_exp.get_metrics(ood_tr_labels, ood_tr_pred_labels)
print('\n** Cross Domain Metrics **')
print('Train Precision: ', p)
print('Train Recall: ', r)
print('Train F1: ', f1)

p, r, f1, _ = t5_exp.get_metrics(ood_te_labels, ood_te_pred_labels)
print('Test Precision: ', p)
print('Test Recall: ', r)
print('Test F1: ', f1)

Model output path:  kevinscaria/ate_tk-instruct-base-def-pos-neg-neut-laptops


Map:   0%|          | 0/3045 [00:00<?, ? examples/s]

Map:   0%|          | 0/800 [00:00<?, ? examples/s]

Map:   0%|          | 0/3041 [00:00<?, ? examples/s]

Map:   0%|          | 0/800 [00:00<?, ? examples/s]

...

Model loaded to:  cuda


100%|██████████| 191/191 [00:54<00:00,  3.51it/s]


Model loaded to:  cuda


100%|██████████| 50/50 [00:15<00:00,  3.13it/s]


** In Domain Metrics **
Train Precision:  0.9105834806769386
Train Recall:  0.9208173690932312
Train F1:  0.9156718313436627
Test Precision:  0.9381746810598626
Test Recall:  0.9263565891472868
Test F1:  0.932228181374939

** Cross Domain Metrics **
Train Precision:  0.8254215304798962
Train Recall:  0.6745813016747934
Train F1:  0.7424171721885209
Test Precision:  0.8497652582159625
Test Recall:  0.6799398948159279
Test F1:  0.7554257095158597


## InstructABSA - 1 - Restaurants 2014

In [ ]:
task_name = 'ate'
model_out_path = 'kevinscaria/ate_tk-instruct-base-def-pos-restaurants'
print('Model output path: ', model_out_path)

# Load the data
id_train_file_path= './Dataset/SemEval14/Train/Restaurants_Train.csv'
id_test_file_path = './Dataset/SemEval14/Test/Restaurants_Test.csv'
id_tr_df = pd.read_csv(id_train_file_path)
id_te_df = pd.read_csv(id_test_file_path)

ood_train_file_path = './Dataset/SemEval14/Train/Laptops_Train.csv'
ood_test_file_path = './Dataset/SemEval14/Test/Laptops_Test.csv'
ood_tr_df = pd.read_csv(ood_train_file_path)
ood_te_df = pd.read_csv(ood_test_file_path)

# Get the input text into the required format using Instructions
instruct_handler = InstructionsHandler()

# Set instruction_set1 for InstructABSA-1 and instruction_set2 for InstructABSA-2
instruct_handler.load_instruction_set1()

# Set bos_instruct1 for lapt14 and bos_instruct2 for rest14. For other datasets, modify the insructions.py file.
loader = DatasetLoader(id_tr_df, id_te_df, ood_tr_df, ood_te_df)
if loader.train_df_id is not None:
    loader.train_df_id = loader.create_data_in_ate_format(loader.train_df_id, 'term', 'raw_text', 'aspectTerms', instruct_handler.ate['bos_instruct2'], instruct_handler.ate['eos_instruct'])
if loader.test_df_id is not None:
    loader.test_df_id = loader.create_data_in_ate_format(loader.test_df_id, 'term', 'raw_text', 'aspectTerms', instruct_handler.ate['bos_instruct2'], instruct_handler.ate['eos_instruct'])

if loader.train_df_ood is not None:
    loader.train_df_ood = loader.create_data_in_ate_format(loader.train_df_ood, 'term', 'raw_text', 'aspectTerms', instruct_handler.ate['bos_instruct1'], instruct_handler.ate['eos_instruct'])
if loader.test_df_ood is not None:
    loader.test_df_ood = loader.create_data_in_ate_format(loader.test_df_ood, 'term', 'raw_text', 'aspectTerms', instruct_handler.ate['bos_instruct1'], instruct_handler.ate['eos_instruct'])

# Model inference - Loading from Checkpoint
t5_exp = T5Generator(model_out_path)

# Tokenize Datasets
id_ds, id_tokenized_ds, ood_ds, ood_tokenized_ds = loader.set_data_for_training_semeval(t5_exp.tokenize_function_inputs)

# Get prediction labels - In Domain Training set   
id_tr_pred_labels = t5_exp.get_labels(tokenized_dataset = id_tokenized_ds, sample_set = 'train', batch_size = 16)
id_tr_labels = [i.strip() for i in id_ds['train']['labels']]

# Get prediction labels - In Domain Testing set
id_te_pred_labels = t5_exp.get_labels(tokenized_dataset = id_tokenized_ds, sample_set = 'test', batch_size = 16)
id_te_labels = [i.strip() for i in id_ds['test']['labels']]

# Get prediction labels - Cross Domain Training set   
ood_tr_pred_labels = t5_exp.get_labels(tokenized_dataset = ood_tokenized_ds, sample_set = 'train', batch_size = 16)
ood_tr_labels = [i.strip() for i in ood_ds['train']['labels']]

# Get prediction labels - Cross Domain Testing set
ood_te_pred_labels = t5_exp.get_labels(tokenized_dataset = ood_tokenized_ds, sample_set = 'test', batch_size = 16)
ood_te_labels = [i.strip() for i in ood_ds['test']['labels']]

p, r, f1, _ = t5_exp.get_metrics(id_tr_labels, id_tr_pred_labels)
print('\n** In Domain Metrics **')
print('Train Precision: ', p)
print('Train Recall: ', r)
print('Train F1: ', f1)

p, r, f1, _ = t5_exp.get_metrics(id_te_labels, id_te_pred_labels)
print('Test Precision: ', p)
print('Test Recall: ', r)
print('Test F1: ', f1)

p, r, f1, _ = t5_exp.get_metrics(ood_tr_labels, ood_tr_pred_labels)
print('\n** Cross Domain Metrics **')
print('Train Precision: ', p)
print('Train Recall: ', r)
print('Train F1: ', f1)

p, r, f1, _ = t5_exp.get_metrics(ood_te_labels, ood_te_pred_labels)
print('Test Precision: ', p)
print('Test Recall: ', r)
print('Test F1: ', f1)

Model output path:  kevinscaria/ate_tk-instruct-base-def-pos-restaurants


Map:   0%|          | 0/3041 [00:00<?, ? examples/s]

Map:   0%|          | 0/800 [00:00<?, ? examples/s]

Map:   0%|          | 0/3045 [00:00<?, ? examples/s]

Map:   0%|          | 0/800 [00:00<?, ? examples/s]

...

Model loaded to:  cuda


100%|██████████| 191/191 [00:47<00:00,  4.05it/s]


Model loaded to:  cuda


100%|██████████| 50/50 [00:12<00:00,  3.95it/s]


** In Domain Metrics **
Train Precision:  0.9271743653766126
Train Recall:  0.9446682213271147
Train F1:  0.9358395463614407
Test Precision:  0.9217134416543574
Test Recall:  0.9376408715251691
Test F1:  0.9296089385474859

** Cross Domain Metrics **
Train Precision:  0.7441741357234315
Train Recall:  0.7422733077905491
Train F1:  0.7432225063938618
Test Precision:  0.7860040567951319
Test Recall:  0.750968992248062
Test F1:  0.7680872150644201


## InstructABSA - 2 - Restaurants 2014

In [ ]:
task_name = 'ate'
model_out_path = 'kevinscaria/ate_tk-instruct-base-def-pos-neg-neut-restaurants'
print('Model output path: ', model_out_path)

# Load the data
id_train_file_path= './Dataset/SemEval14/Train/Restaurants_Train.csv'
id_test_file_path = './Dataset/SemEval14/Test/Restaurants_Test.csv'
id_tr_df = pd.read_csv(id_train_file_path)
id_te_df = pd.read_csv(id_test_file_path)

ood_train_file_path = './Dataset/SemEval14/Train/Laptops_Train.csv'
ood_test_file_path = './Dataset/SemEval14/Test/Laptops_Test.csv'
ood_tr_df = pd.read_csv(ood_train_file_path)
ood_te_df = pd.read_csv(ood_test_file_path)

# Get the input text into the required format using Instructions
instruct_handler = InstructionsHandler()

# Set instruction_set1 for InstructABSA-1 and instruction_set2 for InstructABSA-2
instruct_handler.load_instruction_set2()

# Set bos_instruct1 for lapt14 and bos_instruct2 for rest14. For other datasets, modify the insructions.py file.
loader = DatasetLoader(id_tr_df, id_te_df, ood_tr_df, ood_te_df)
if loader.train_df_id is not None:
    loader.train_df_id = loader.create_data_in_ate_format(loader.train_df_id, 'term', 'raw_text', 'aspectTerms', instruct_handler.ate['bos_instruct2'], instruct_handler.ate['eos_instruct'])
if loader.test_df_id is not None:
    loader.test_df_id = loader.create_data_in_ate_format(loader.test_df_id, 'term', 'raw_text', 'aspectTerms', instruct_handler.ate['bos_instruct2'], instruct_handler.ate['eos_instruct'])

if loader.train_df_ood is not None:
    loader.train_df_ood = loader.create_data_in_ate_format(loader.train_df_ood, 'term', 'raw_text', 'aspectTerms', instruct_handler.ate['bos_instruct1'], instruct_handler.ate['eos_instruct'])
if loader.test_df_ood is not None:
    loader.test_df_ood = loader.create_data_in_ate_format(loader.test_df_ood, 'term', 'raw_text', 'aspectTerms', instruct_handler.ate['bos_instruct1'], instruct_handler.ate['eos_instruct'])

# Model inference - Loading from Checkpoint
t5_exp = T5Generator(model_out_path)

# Tokenize Datasets
id_ds, id_tokenized_ds, ood_ds, ood_tokenized_ds = loader.set_data_for_training_semeval(t5_exp.tokenize_function_inputs)

# Get prediction labels - In Domain Training set   
id_tr_pred_labels = t5_exp.get_labels(tokenized_dataset = id_tokenized_ds, sample_set = 'train', batch_size = 16)
id_tr_labels = [i.strip() for i in id_ds['train']['labels']]

# Get prediction labels - In Domain Testing set
id_te_pred_labels = t5_exp.get_labels(tokenized_dataset = id_tokenized_ds, sample_set = 'test', batch_size = 16)
id_te_labels = [i.strip() for i in id_ds['test']['labels']]

# Get prediction labels - Cross Domain Training set   
ood_tr_pred_labels = t5_exp.get_labels(tokenized_dataset = ood_tokenized_ds, sample_set = 'train', batch_size = 16)
ood_tr_labels = [i.strip() for i in ood_ds['train']['labels']]

# Get prediction labels - Cross Domain Testing set
ood_te_pred_labels = t5_exp.get_labels(tokenized_dataset = ood_tokenized_ds, sample_set = 'test', batch_size = 16)
ood_te_labels = [i.strip() for i in ood_ds['test']['labels']]

p, r, f1, _ = t5_exp.get_metrics(id_tr_labels, id_tr_pred_labels)
print('\n** In Domain Metrics **')
print('Train Precision: ', p)
print('Train Recall: ', r)
print('Train F1: ', f1)

p, r, f1, _ = t5_exp.get_metrics(id_te_labels, id_te_pred_labels)
print('Test Precision: ', p)
print('Test Recall: ', r)
print('Test F1: ', f1)

p, r, f1, _ = t5_exp.get_metrics(ood_tr_labels, ood_tr_pred_labels)
print('\n** Cross Domain Metrics **')
print('Train Precision: ', p)
print('Train Recall: ', r)
print('Train F1: ', f1)

p, r, f1, _ = t5_exp.get_metrics(ood_te_labels, ood_te_pred_labels)
print('Test Precision: ', p)
print('Test Recall: ', r)
print('Test F1: ', f1)

Model output path:  kevinscaria/ate_tk-instruct-base-def-pos-neg-neut-restaurants


Map:   0%|          | 0/3041 [00:00<?, ? examples/s]

Map:   0%|          | 0/800 [00:00<?, ? examples/s]

Map:   0%|          | 0/3045 [00:00<?, ? examples/s]

Map:   0%|          | 0/800 [00:00<?, ? examples/s]

...

Model loaded to:  cuda


100%|██████████| 191/191 [00:46<00:00,  4.08it/s]


Model loaded to:  cuda


100%|██████████| 50/50 [00:12<00:00,  4.10it/s]


** In Domain Metrics **
Train Precision:  0.9626684338413961
Train Recall:  0.9238923044307823
Train F1:  0.9428818693206406
Test Precision:  0.9449185415050426
Test Recall:  0.9151014274981217
Test F1:  0.9297709923664123

** Cross Domain Metrics **
Train Precision:  0.802646085997795
Train Recall:  0.7438058748403577
Train F1:  0.772106588890362
Test Precision:  0.8180851063829787
Test Recall:  0.7451550387596899
Test F1:  0.7799188640973631


# Aspect Term Sentiment Classification

## InstructABSA - 1 - Laptops 2014

In [ ]:
task_name = 'atsc'
model_out_path = 'kevinscaria/atsc_tk-instruct-base-def-pos-laptops'
print('Model output path: ', model_out_path)

# Load the data
id_train_file_path = './Dataset/SemEval14/Train/Laptops_Train.csv'
id_test_file_path = './Dataset/SemEval14/Test/Laptops_Test.csv'
id_tr_df = pd.read_csv(id_train_file_path)
id_te_df = pd.read_csv(id_test_file_path)

ood_train_file_path = './Dataset/SemEval14/Train/Restaurants_Train.csv'
ood_test_file_path = './Dataset/SemEval14/Test/Restaurants_Test.csv'
ood_tr_df = pd.read_csv(ood_train_file_path)
ood_te_df = pd.read_csv(ood_test_file_path)

# Get the input text into the required format using Instructions
instruct_handler = InstructionsHandler()

# Set instruction_set1 for InstructABSA-1 and instruction_set2 for InstructABSA-2
instruct_handler.load_instruction_set1()

loader = DatasetLoader(id_tr_df, id_te_df, ood_tr_df, ood_te_df)
# Set bos_instruct1 for lapt14 and bos_instruct2 for rest14. For other datasets, modify the insructions.py file.
if loader.train_df_id is not None:
    loader.train_df_id = loader.create_data_in_atsc_format(loader.train_df_id, 'aspectTerms', 'term', 'raw_text', 'aspect', instruct_handler.atsc['bos_instruct1'], 
                                                           instruct_handler.atsc['delim_instruct'], instruct_handler.atsc['eos_instruct'])
if loader.test_df_id is not None:
    loader.test_df_id = loader.create_data_in_atsc_format(loader.test_df_id, 'aspectTerms', 'term', 'raw_text', 'aspect', instruct_handler.atsc['bos_instruct1'], 
                                                           instruct_handler.atsc['delim_instruct'], instruct_handler.atsc['eos_instruct'])
if loader.train_df_ood is not None:
    loader.train_df_ood = loader.create_data_in_atsc_format(loader.train_df_ood, 'aspectTerms', 'term', 'raw_text', 'aspect', instruct_handler.atsc['bos_instruct2'], 
                                                           instruct_handler.atsc['delim_instruct'], instruct_handler.atsc['eos_instruct'])
if loader.test_df_ood is not None:
    loader.test_df_ood = loader.create_data_in_atsc_format(loader.test_df_ood, 'aspectTerms', 'term', 'raw_text', 'aspect', instruct_handler.atsc['bos_instruct2'], 
                                                           instruct_handler.atsc['delim_instruct'], instruct_handler.atsc['eos_instruct'])
    
# Data preprocess
loader.train_df_id  = loader.train_df_id[loader.train_df_id['labels'] != 'none']
loader.test_df_id  = loader.test_df_id[loader.test_df_id['labels'] != 'none']
loader.train_df_ood  = loader.train_df_ood[loader.train_df_ood['labels'] != 'none']
loader.test_df_ood  = loader.test_df_ood[loader.test_df_ood['labels'] != 'none']

loader.train_df_id  = loader.train_df_id[loader.train_df_id['labels'] != 'conflict']
loader.test_df_id  = loader.test_df_id[loader.test_df_id['labels'] != 'conflict']
loader.train_df_ood  = loader.train_df_ood[loader.train_df_ood['labels'] != 'conflict']
loader.test_df_ood  = loader.test_df_ood[loader.test_df_ood['labels'] != 'conflict']


# Model inference - Loading from Checkpoint
t5_exp = T5Classifier(model_out_path)

# Tokenize Datasets
id_ds, id_tokenized_ds, ood_ds, ood_tokenized_ds = loader.set_data_for_training_semeval(t5_exp.tokenize_function_inputs)

# Get prediction labels - In Domain Training set   
id_tr_pred_labels = t5_exp.get_labels(tokenized_dataset = id_tokenized_ds, sample_set = 'train', batch_size = 16)
id_tr_labels = [i.strip() for i in id_ds['train']['labels']]

# Get prediction labels - In Domain Testing set
id_te_pred_labels = t5_exp.get_labels(tokenized_dataset = id_tokenized_ds, sample_set = 'test', batch_size = 16)
id_te_labels = [i.strip() for i in id_ds['test']['labels']]

# Get prediction labels - Cross Domain Training set   
ood_tr_pred_labels = t5_exp.get_labels(tokenized_dataset = ood_tokenized_ds, sample_set = 'train', batch_size = 16)
ood_tr_labels = [i.strip() for i in ood_ds['train']['labels']]

# Get prediction labels - Cross Domain Testing set
ood_te_pred_labels = t5_exp.get_labels(tokenized_dataset = ood_tokenized_ds, sample_set = 'test', batch_size = 16)
ood_te_labels = [i.strip() for i in ood_ds['test']['labels']]

p, r, f1, acc = t5_exp.get_metrics(id_tr_labels, id_tr_pred_labels)
print('\n** In Domain Metrics **')
print('Train Precision: ', p)
print('Train Recall: ', r)
print('Train F1: ', f1)
print('Train Accuracy: ', acc)

p, r, f1, acc = t5_exp.get_metrics(id_te_labels, id_te_pred_labels)
print('Test Precision: ', p)
print('Test Recall: ', r)
print('Test F1: ', f1)
print('Test Accuracy: ', acc)

p, r, f1, acc = t5_exp.get_metrics(ood_tr_labels, ood_tr_pred_labels)
print('\n** Cross Domain Metrics **')
print('Train Precision: ', p)
print('Train Recall: ', r)
print('Train F1: ', f1)
print('Train Accuracy: ', acc)

p, r, f1, acc = t5_exp.get_metrics(ood_te_labels, ood_te_pred_labels)
print('Test Precision: ', p)
print('Test Recall: ', r)
print('Test F1: ', f1)
print('Test Accuracy: ', acc)

Model output path:  kevinscaria/atsc_tk-instruct-base-def-pos-laptops


Map:   0%|          | 0/2313 [00:00<?, ? examples/s]

Map:   0%|          | 0/638 [00:00<?, ? examples/s]

Map:   0%|          | 0/3602 [00:00<?, ? examples/s]

Map:   0%|          | 0/1120 [00:00<?, ? examples/s]

...

Prediction from checkpoint
Model loaded to:  cuda


100%|██████████| 226/226 [00:58<00:00,  3.85it/s]


Prediction from checkpoint
Model loaded to:  cuda


100%|██████████| 70/70 [00:18<00:00,  3.76it/s]


** In Domain Metrics **
Train Precision:  0.700747141650441
Train Recall:  0.7039761515119792
Train F1:  0.7022748478396899
Train Accuracy:  0.9463899697362732
Test Precision:  0.797203570011558
Test Recall:  0.8101647951407567
Test F1:  0.7937173337855349
Test Accuracy:  0.829153605015674

** Cross Domain Metrics **
Train Precision:  0.5078728131757032
Train Recall:  0.5371342339347723
Train F1:  0.5148180207786444
Train Accuracy:  0.7357023875624653
Test Precision:  0.7361763969850355
Test Recall:  0.7986656200941916
Test F1:  0.7595471749177545
Test Accuracy:  0.8133928571428571


## InstructABSA - 2 - Laptops 2014

In [ ]:
task_name = 'atsc'
model_out_path = 'kevinscaria/atsc_tk-instruct-base-def-pos-neg-neut-laptops'
print('Model output path: ', model_out_path)
 
# Load the data
id_train_file_path = './Dataset/SemEval14/Train/Laptops_Train.csv'
id_test_file_path = './Dataset/SemEval14/Test/Laptops_Test.csv'
id_tr_df = pd.read_csv(id_train_file_path)
id_te_df = pd.read_csv(id_test_file_path)

ood_train_file_path = './Dataset/SemEval14/Train/Restaurants_Train.csv'
ood_test_file_path = './Dataset/SemEval14/Test/Restaurants_Test.csv'
ood_tr_df = pd.read_csv(ood_train_file_path)
ood_te_df = pd.read_csv(ood_test_file_path)

# Get the input text into the required format using Instructions
instruct_handler = InstructionsHandler()

# Set instruction_set1 for InstructABSA-1 and instruction_set2 for InstructABSA-2
instruct_handler.load_instruction_set2()

loader = DatasetLoader(id_tr_df, id_te_df, ood_tr_df, ood_te_df)
# Set bos_instruct1 for lapt14 and bos_instruct2 for rest14. For other datasets, modify the insructions.py file.
if loader.train_df_id is not None:
    loader.train_df_id = loader.create_data_in_atsc_format(loader.train_df_id, 'aspectTerms', 'term', 'raw_text', 'aspect', instruct_handler.atsc['bos_instruct1'], 
                                                           instruct_handler.atsc['delim_instruct'], instruct_handler.atsc['eos_instruct'])
if loader.test_df_id is not None:
    loader.test_df_id = loader.create_data_in_atsc_format(loader.test_df_id, 'aspectTerms', 'term', 'raw_text', 'aspect', instruct_handler.atsc['bos_instruct1'], 
                                                           instruct_handler.atsc['delim_instruct'], instruct_handler.atsc['eos_instruct'])
if loader.train_df_ood is not None:
    loader.train_df_ood = loader.create_data_in_atsc_format(loader.train_df_ood, 'aspectTerms', 'term', 'raw_text', 'aspect', instruct_handler.atsc['bos_instruct2'], 
                                                           instruct_handler.atsc['delim_instruct'], instruct_handler.atsc['eos_instruct'])
if loader.test_df_ood is not None:
    loader.test_df_ood = loader.create_data_in_atsc_format(loader.test_df_ood, 'aspectTerms', 'term', 'raw_text', 'aspect', instruct_handler.atsc['bos_instruct2'], 
                                                           instruct_handler.atsc['delim_instruct'], instruct_handler.atsc['eos_instruct'])
    
# Data preprocess
loader.train_df_id  = loader.train_df_id[loader.train_df_id['labels'] != 'none']
loader.test_df_id  = loader.test_df_id[loader.test_df_id['labels'] != 'none']
loader.train_df_ood  = loader.train_df_ood[loader.train_df_ood['labels'] != 'none']
loader.test_df_ood  = loader.test_df_ood[loader.test_df_ood['labels'] != 'none']

loader.train_df_id  = loader.train_df_id[loader.train_df_id['labels'] != 'conflict']
loader.test_df_id  = loader.test_df_id[loader.test_df_id['labels'] != 'conflict']
loader.train_df_ood  = loader.train_df_ood[loader.train_df_ood['labels'] != 'conflict']
loader.test_df_ood  = loader.test_df_ood[loader.test_df_ood['labels'] != 'conflict']


# Model inference - Loading from Checkpoint
t5_exp = T5Classifier(model_out_path)

# Tokenize Datasets
id_ds, id_tokenized_ds, ood_ds, ood_tokenized_ds = loader.set_data_for_training_semeval(t5_exp.tokenize_function_inputs)

# Get prediction labels - In Domain Training set   
id_tr_pred_labels = t5_exp.get_labels(tokenized_dataset = id_tokenized_ds, sample_set = 'train', batch_size = 16)
id_tr_labels = [i.strip() for i in id_ds['train']['labels']]

# Get prediction labels - In Domain Testing set
id_te_pred_labels = t5_exp.get_labels(tokenized_dataset = id_tokenized_ds, sample_set = 'test', batch_size = 16)
id_te_labels = [i.strip() for i in id_ds['test']['labels']]

# Get prediction labels - Cross Domain Training set   
ood_tr_pred_labels = t5_exp.get_labels(tokenized_dataset = ood_tokenized_ds, sample_set = 'train', batch_size = 16)
ood_tr_labels = [i.strip() for i in ood_ds['train']['labels']]

# Get prediction labels - Cross Domain Testing set
ood_te_pred_labels = t5_exp.get_labels(tokenized_dataset = ood_tokenized_ds, sample_set = 'test', batch_size = 16)
ood_te_labels = [i.strip() for i in ood_ds['test']['labels']]

p, r, f1, acc = t5_exp.get_metrics(id_tr_labels, id_tr_pred_labels)
print('\n** In Domain Metrics **')
print('Train Precision: ', p)
print('Train Recall: ', r)
print('Train F1: ', f1)
print('Train Accuracy: ', acc)

p, r, f1, acc = t5_exp.get_metrics(id_te_labels, id_te_pred_labels)
print('Test Precision: ', p)
print('Test Recall: ', r)
print('Test F1: ', f1)
print('Test Accuracy: ', acc)

p, r, f1, acc = t5_exp.get_metrics(ood_tr_labels, ood_tr_pred_labels)
print('\n** Cross Domain Metrics **')
print('Train Precision: ', p)
print('Train Recall: ', r)
print('Train F1: ', f1)
print('Train Accuracy: ', acc)

p, r, f1, acc = t5_exp.get_metrics(ood_te_labels, ood_te_pred_labels)
print('Test Precision: ', p)
print('Test Recall: ', r)
print('Test F1: ', f1)
print('Test Accuracy: ', acc)

Model output path:  kevinscaria/atsc_tk-instruct-base-def-pos-neg-neut-laptops


Map:   0%|          | 0/2313 [00:00<?, ? examples/s]

Map:   0%|          | 0/638 [00:00<?, ? examples/s]

Map:   0%|          | 0/3602 [00:00<?, ? examples/s]

Map:   0%|          | 0/1120 [00:00<?, ? examples/s]

...

Prediction from checkpoint
Model loaded to:  cuda


100%|██████████| 226/226 [01:51<00:00,  2.02it/s]


Prediction from checkpoint
Model loaded to:  cuda


100%|██████████| 70/70 [00:34<00:00,  2.00it/s]


** In Domain Metrics **
Train Precision:  0.8586201769727722
Train Recall:  0.8309856276997086
Train F1:  0.8404014941683887
Train Accuracy:  0.8715953307392996
Test Precision:  0.7840452860537691
Test Recall:  0.7718139684447066
Test F1:  0.7578312764137811
Test Accuracy:  0.8056426332288401

** Cross Domain Metrics **
Train Precision:  0.6521618963108519
Train Recall:  0.6722118219798446
Train F1:  0.6530490773183615
Train Accuracy:  0.7470849528039978
Test Precision:  0.7041642633862081
Test Recall:  0.7251439037153323
Test F1:  0.7003373297759087
Test Accuracy:  0.8008928571428572


## InstructABSA - 1 - Restaurants 2014

In [ ]:
task_name = 'atsc'
model_out_path = 'kevinscaria/atsc_tk-instruct-base-def-pos-restaurants'
print('Model output path: ', model_out_path)

# Load the data
id_train_file_path = './Dataset/SemEval14/Train/Restaurants_Train.csv'
id_test_file_path = './Dataset/SemEval14/Test/Restaurants_Test.csv'
id_tr_df = pd.read_csv(id_train_file_path)
id_te_df = pd.read_csv(id_test_file_path)

ood_train_file_path = './Dataset/SemEval14/Train/Laptops_Train.csv'
ood_test_file_path = './Dataset/SemEval14/Test/Laptops_Test.csv'
ood_tr_df = pd.read_csv(ood_train_file_path)
ood_te_df = pd.read_csv(ood_test_file_path)

# Get the input text into the required format using Instructions
instruct_handler = InstructionsHandler()

# Set instruction_set1 for InstructABSA-1 and instruction_set2 for InstructABSA-2
instruct_handler.load_instruction_set1()

loader = DatasetLoader(id_tr_df, id_te_df, ood_tr_df, ood_te_df)
# Set bos_instruct1 for lapt14 and bos_instruct2 for rest14. For other datasets, modify the insructions.py file.
if loader.train_df_id is not None:
    loader.train_df_id = loader.create_data_in_atsc_format(loader.train_df_id, 'aspectTerms', 'term', 'raw_text', 'aspect', instruct_handler.atsc['bos_instruct2'], 
                                                           instruct_handler.atsc['delim_instruct'], instruct_handler.atsc['eos_instruct'])
if loader.test_df_id is not None:
    loader.test_df_id = loader.create_data_in_atsc_format(loader.test_df_id, 'aspectTerms', 'term', 'raw_text', 'aspect', instruct_handler.atsc['bos_instruct2'], 
                                                           instruct_handler.atsc['delim_instruct'], instruct_handler.atsc['eos_instruct'])
if loader.train_df_ood is not None:
    loader.train_df_ood = loader.create_data_in_atsc_format(loader.train_df_ood, 'aspectTerms', 'term', 'raw_text', 'aspect', instruct_handler.atsc['bos_instruct1'], 
                                                           instruct_handler.atsc['delim_instruct'], instruct_handler.atsc['eos_instruct'])
if loader.test_df_ood is not None:
    loader.test_df_ood = loader.create_data_in_atsc_format(loader.test_df_ood, 'aspectTerms', 'term', 'raw_text', 'aspect', instruct_handler.atsc['bos_instruct1'], 
                                                           instruct_handler.atsc['delim_instruct'], instruct_handler.atsc['eos_instruct'])
    
# Data preprocess
loader.train_df_id  = loader.train_df_id[loader.train_df_id['labels'] != 'none']
loader.test_df_id  = loader.test_df_id[loader.test_df_id['labels'] != 'none']
loader.train_df_ood  = loader.train_df_ood[loader.train_df_ood['labels'] != 'none']
loader.test_df_ood  = loader.test_df_ood[loader.test_df_ood['labels'] != 'none']

loader.train_df_id  = loader.train_df_id[loader.train_df_id['labels'] != 'conflict']
loader.test_df_id  = loader.test_df_id[loader.test_df_id['labels'] != 'conflict']
loader.train_df_ood  = loader.train_df_ood[loader.train_df_ood['labels'] != 'conflict']
loader.test_df_ood  = loader.test_df_ood[loader.test_df_ood['labels'] != 'conflict']


# Model inference - Loading from Checkpoint
t5_exp = T5Classifier(model_out_path)

# Tokenize Datasets
id_ds, id_tokenized_ds, ood_ds, ood_tokenized_ds = loader.set_data_for_training_semeval(t5_exp.tokenize_function_inputs)

# Get prediction labels - In Domain Training set   
id_tr_pred_labels = t5_exp.get_labels(tokenized_dataset = id_tokenized_ds, sample_set = 'train', batch_size = 16)
id_tr_labels = [i.strip() for i in id_ds['train']['labels']]

# Get prediction labels - In Domain Testing set
id_te_pred_labels = t5_exp.get_labels(tokenized_dataset = id_tokenized_ds, sample_set = 'test', batch_size = 16)
id_te_labels = [i.strip() for i in id_ds['test']['labels']]

# Get prediction labels - Cross Domain Training set   
ood_tr_pred_labels = t5_exp.get_labels(tokenized_dataset = ood_tokenized_ds, sample_set = 'train', batch_size = 16)
ood_tr_labels = [i.strip() for i in ood_ds['train']['labels']]

# Get prediction labels - Cross Domain Testing set
ood_te_pred_labels = t5_exp.get_labels(tokenized_dataset = ood_tokenized_ds, sample_set = 'test', batch_size = 16)
ood_te_labels = [i.strip() for i in ood_ds['test']['labels']]

p, r, f1, acc = t5_exp.get_metrics(id_tr_labels, id_tr_pred_labels)
print('\n** In Domain Metrics **')
print('Train Precision: ', p)
print('Train Recall: ', r)
print('Train F1: ', f1)
print('Train Accuracy: ', acc)

p, r, f1, acc = t5_exp.get_metrics(id_te_labels, id_te_pred_labels)
print('Test Precision: ', p)
print('Test Recall: ', r)
print('Test F1: ', f1)
print('Test Accuracy: ', acc)

p, r, f1, acc = t5_exp.get_metrics(ood_tr_labels, ood_tr_pred_labels)
print('\n** Cross Domain Metrics **')
print('Train Precision: ', p)
print('Train Recall: ', r)
print('Train F1: ', f1)
print('Train Accuracy: ', acc)

p, r, f1, acc = t5_exp.get_metrics(ood_te_labels, ood_te_pred_labels)
print('Test Precision: ', p)
print('Test Recall: ', r)
print('Test F1: ', f1)
print('Test Accuracy: ', acc)

Model output path:  kevinscaria/atsc_tk-instruct-base-def-pos-restaurants


Map:   0%|          | 0/3602 [00:00<?, ? examples/s]

Map:   0%|          | 0/1120 [00:00<?, ? examples/s]

Map:   0%|          | 0/2313 [00:00<?, ? examples/s]

Map:   0%|          | 0/638 [00:00<?, ? examples/s]

...

Prediction from checkpoint
Model loaded to:  cuda


100%|██████████| 145/145 [00:41<00:00,  3.47it/s]


Prediction from checkpoint
Model loaded to:  cuda


100%|██████████| 40/40 [00:11<00:00,  3.53it/s]


** In Domain Metrics **
Train Precision:  0.6413790136840789
Train Recall:  0.6335273079047725
Train F1:  0.6370797844211663
Train Accuracy:  0.8925596890616324
Test Precision:  0.8220170679864217
Test Recall:  0.7704081632653063
Test F1:  0.7899600712171111
Test Accuracy:  0.8625

** Cross Domain Metrics **
Train Precision:  0.7336501887448881
Train Recall:  0.725974068825718
Train F1:  0.7234356125265432
Train Accuracy:  0.7630782533506268
Test Precision:  0.8112078846471799
Test Recall:  0.7813425911867289
Test F1:  0.7927080610021786
Test Accuracy:  0.829153605015674


## InstructABSA - 2 - Restaurants 2014

In [ ]:
task_name = 'atsc'
model_out_path = 'kevinscaria/atsc_tk-instruct-base-def-pos-neg-neut-restaurants'
print('Model output path: ', model_out_path)
 
# Load the data
id_train_file_path = './Dataset/SemEval14/Train/Restaurants_Train.csv'
id_test_file_path = './Dataset/SemEval14/Test/Restaurants_Test.csv'
id_tr_df = pd.read_csv(id_train_file_path)
id_te_df = pd.read_csv(id_test_file_path)

ood_train_file_path = './Dataset/SemEval14/Train/Laptops_Train.csv'
ood_test_file_path = './Dataset/SemEval14/Test/Laptops_Test.csv'
ood_tr_df = pd.read_csv(ood_train_file_path)
ood_te_df = pd.read_csv(ood_test_file_path)

# Get the input text into the required format using Instructions
instruct_handler = InstructionsHandler()

# Set instruction_set1 for InstructABSA-1 and instruction_set2 for InstructABSA-2
instruct_handler.load_instruction_set2()

loader = DatasetLoader(id_tr_df, id_te_df, ood_tr_df, ood_te_df)
# Set bos_instruct1 for lapt14 and bos_instruct2 for rest14. For other datasets, modify the insructions.py file.
if loader.train_df_id is not None:
    loader.train_df_id = loader.create_data_in_atsc_format(loader.train_df_id, 'aspectTerms', 'term', 'raw_text', 'aspect', instruct_handler.atsc['bos_instruct2'], 
                                                           instruct_handler.atsc['delim_instruct'], instruct_handler.atsc['eos_instruct'])
if loader.test_df_id is not None:
    loader.test_df_id = loader.create_data_in_atsc_format(loader.test_df_id, 'aspectTerms', 'term', 'raw_text', 'aspect', instruct_handler.atsc['bos_instruct2'], 
                                                           instruct_handler.atsc['delim_instruct'], instruct_handler.atsc['eos_instruct'])
if loader.train_df_ood is not None:
    loader.train_df_ood = loader.create_data_in_atsc_format(loader.train_df_ood, 'aspectTerms', 'term', 'raw_text', 'aspect', instruct_handler.atsc['bos_instruct1'], 
                                                           instruct_handler.atsc['delim_instruct'], instruct_handler.atsc['eos_instruct'])
if loader.test_df_ood is not None:
    loader.test_df_ood = loader.create_data_in_atsc_format(loader.test_df_ood, 'aspectTerms', 'term', 'raw_text', 'aspect', instruct_handler.atsc['bos_instruct1'], 
                                                           instruct_handler.atsc['delim_instruct'], instruct_handler.atsc['eos_instruct'])
    
# Data preprocess
loader.train_df_id  = loader.train_df_id[loader.train_df_id['labels'] != 'none']
loader.test_df_id  = loader.test_df_id[loader.test_df_id['labels'] != 'none']
loader.train_df_ood  = loader.train_df_ood[loader.train_df_ood['labels'] != 'none']
loader.test_df_ood  = loader.test_df_ood[loader.test_df_ood['labels'] != 'none']

loader.train_df_id  = loader.train_df_id[loader.train_df_id['labels'] != 'conflict']
loader.test_df_id  = loader.test_df_id[loader.test_df_id['labels'] != 'conflict']
loader.train_df_ood  = loader.train_df_ood[loader.train_df_ood['labels'] != 'conflict']
loader.test_df_ood  = loader.test_df_ood[loader.test_df_ood['labels'] != 'conflict']


# Model inference - Loading from Checkpoint
t5_exp = T5Classifier(model_out_path)

# Tokenize Datasets
id_ds, id_tokenized_ds, ood_ds, ood_tokenized_ds = loader.set_data_for_training_semeval(t5_exp.tokenize_function_inputs)

# Get prediction labels - In Domain Training set   
id_tr_pred_labels = t5_exp.get_labels(tokenized_dataset = id_tokenized_ds, sample_set = 'train', batch_size = 16)
id_tr_labels = [i.strip() for i in id_ds['train']['labels']]

# Get prediction labels - In Domain Testing set
id_te_pred_labels = t5_exp.get_labels(tokenized_dataset = id_tokenized_ds, sample_set = 'test', batch_size = 16)
id_te_labels = [i.strip() for i in id_ds['test']['labels']]

# Get prediction labels - Cross Domain Training set   
ood_tr_pred_labels = t5_exp.get_labels(tokenized_dataset = ood_tokenized_ds, sample_set = 'train', batch_size = 16)
ood_tr_labels = [i.strip() for i in ood_ds['train']['labels']]

# Get prediction labels - Cross Domain Testing set
ood_te_pred_labels = t5_exp.get_labels(tokenized_dataset = ood_tokenized_ds, sample_set = 'test', batch_size = 16)
ood_te_labels = [i.strip() for i in ood_ds['test']['labels']]

p, r, f1, acc = t5_exp.get_metrics(id_tr_labels, id_tr_pred_labels)
print('\n** In Domain Metrics **')
print('Train Precision: ', p)
print('Train Recall: ', r)
print('Train F1: ', f1)
print('Train Accuracy: ', acc)

p, r, f1, acc = t5_exp.get_metrics(id_te_labels, id_te_pred_labels)
print('Test Precision: ', p)
print('Test Recall: ', r)
print('Test F1: ', f1)
print('Test Accuracy: ', acc)

p, r, f1, acc = t5_exp.get_metrics(ood_tr_labels, ood_tr_pred_labels)
print('\n** Cross Domain Metrics **')
print('Train Precision: ', p)
print('Train Recall: ', r)
print('Train F1: ', f1)
print('Train Accuracy: ', acc)

p, r, f1, acc = t5_exp.get_metrics(ood_te_labels, ood_te_pred_labels)
print('Test Precision: ', p)
print('Test Recall: ', r)
print('Test F1: ', f1)
print('Test Accuracy: ', acc)

Model output path:  kevinscaria/atsc_tk-instruct-base-def-pos-neg-neut-restaurants


Map:   0%|          | 0/3602 [00:00<?, ? examples/s]

Map:   0%|          | 0/1120 [00:00<?, ? examples/s]

Map:   0%|          | 0/2313 [00:00<?, ? examples/s]

Map:   0%|          | 0/638 [00:00<?, ? examples/s]

...

Prediction from checkpoint
Model loaded to:  cuda


100%|██████████| 145/145 [01:07<00:00,  2.16it/s]


Prediction from checkpoint
Model loaded to:  cuda


100%|██████████| 40/40 [00:17<00:00,  2.24it/s]


** In Domain Metrics **
Train Precision:  0.8084817850637522
Train Recall:  0.8398896863324987
Train F1:  0.8221526639628643
Train Accuracy:  0.8617434758467518
Test Precision:  0.7894451812555262
Test Recall:  0.7906855049712194
Test F1:  0.7859729615478853
Test Accuracy:  0.8535714285714285

** Cross Domain Metrics **
Train Precision:  0.3682812831168804
Train Recall:  0.3691724147356068
Train F1:  0.36698838349918006
Train Accuracy:  0.7674016428880243
Test Precision:  0.8036160138099505
Test Recall:  0.8082145509060831
Test F1:  0.804614192486046
Test Accuracy:  0.835423197492163


# Joint Task - Aspect Term and Polarity Co Extraction

## InstructABSA - 1 - Laptops 2014

In [ ]:
task_name = 'joint_task'
model_out_path = 'kevinscaria/joint_tk-instruct-base-def-pos-laptops'
print('Model output path: ', model_out_path)
 
# Load the data
id_train_file_path = './Dataset/SemEval14/Train/Laptops_Train.csv'
id_test_file_path = './Dataset/SemEval14/Test/Laptops_Test.csv'
id_tr_df = pd.read_csv(id_train_file_path)
id_te_df = pd.read_csv(id_test_file_path)

ood_train_file_path = './Dataset/SemEval14/Train/Restaurants_Train.csv'
ood_test_file_path = './Dataset/SemEval14/Test/Restaurants_Test.csv'
ood_tr_df = pd.read_csv(ood_train_file_path)
ood_te_df = pd.read_csv(ood_test_file_path)

# Get the input text into the required format using Instructions
instruct_handler = InstructionsHandler()

# Set instruction_set1 for InstructABSA-1 and instruction_set2 for InstructABSA-2
instruct_handler.load_instruction_set1()

loader = DatasetLoader(id_tr_df, id_te_df, ood_tr_df, ood_te_df)
# Set bos_instruct1 for lapt14 and bos_instruct2 for rest14. For other datasets, modify the insructions.py file.
if loader.train_df_id is not None:
    loader.train_df_id = loader.create_data_in_joint_task_format(loader.train_df_id, 'term', 'polarity', 'raw_text', 'aspectTerms', instruct_handler.joint['bos_instruct1'], instruct_handler.joint['eos_instruct'])
if loader.test_df_id is not None:
    loader.test_df_id = loader.create_data_in_joint_task_format(loader.test_df_id, 'term', 'polarity', 'raw_text', 'aspectTerms', instruct_handler.joint['bos_instruct1'], instruct_handler.joint['eos_instruct'])
if loader.train_df_ood is not None:
    loader.train_df_ood = loader.create_data_in_joint_task_format(loader.train_df_ood, 'term', 'polarity', 'raw_text', 'aspectTerms', instruct_handler.joint['bos_instruct2'], instruct_handler.joint['eos_instruct'])
if loader.test_df_ood is not None:
    loader.test_df_ood = loader.create_data_in_joint_task_format(loader.test_df_ood, 'term', 'polarity', 'raw_text', 'aspectTerms', instruct_handler.joint['bos_instruct2'], instruct_handler.joint['eos_instruct'])    
    
# Model inference - Loading from Checkpoint
t5_exp = T5Generator(model_out_path)

# Tokenize Datasets
id_ds, id_tokenized_ds, ood_ds, ood_tokenized_ds = loader.set_data_for_training_semeval(t5_exp.tokenize_function_inputs)

# Get prediction labels - In Domain Training set   
id_tr_pred_labels = t5_exp.get_labels(tokenized_dataset = id_tokenized_ds, sample_set = 'train', batch_size = 16)
id_tr_labels = [i.strip() for i in id_ds['train']['labels']]

# Get prediction labels - In Domain Testing set
id_te_pred_labels = t5_exp.get_labels(tokenized_dataset = id_tokenized_ds, sample_set = 'test', batch_size = 16)
id_te_labels = [i.strip() for i in id_ds['test']['labels']]

# Get prediction labels - Cross Domain Training set   
ood_tr_pred_labels = t5_exp.get_labels(tokenized_dataset = ood_tokenized_ds, sample_set = 'train', batch_size = 16)
ood_tr_labels = [i.strip() for i in ood_ds['train']['labels']]

# Get prediction labels - Cross Domain Testing set
ood_te_pred_labels = t5_exp.get_labels(tokenized_dataset = ood_tokenized_ds, sample_set = 'test', batch_size = 16)
ood_te_labels = [i.strip() for i in ood_ds['test']['labels']]

p, r, f1, _ = t5_exp.get_metrics(id_tr_labels, id_tr_pred_labels)
print('\n** In Domain Metrics **')
print('Train Precision: ', p)
print('Train Recall: ', r)
print('Train F1: ', f1)

p, r, f1, _ = t5_exp.get_metrics(id_te_labels, id_te_pred_labels)
print('Test Precision: ', p)
print('Test Recall: ', r)
print('Test F1: ', f1)

p, r, f1, _ = t5_exp.get_metrics(ood_tr_labels, ood_tr_pred_labels)
print('\n** Cross Domain Metrics **')
print('Train Precision: ', p)
print('Train Recall: ', r)
print('Train F1: ', f1)

p, r, f1, _ = t5_exp.get_metrics(ood_te_labels, ood_te_pred_labels)
print('Test Precision: ', p)
print('Test Recall: ', r)
print('Test F1: ', f1)

Model output path:  kevinscaria/joint_tk-instruct-base-def-pos-laptops


Map:   0%|          | 0/3045 [00:00<?, ? examples/s]

Map:   0%|          | 0/800 [00:00<?, ? examples/s]

Map:   0%|          | 0/3041 [00:00<?, ? examples/s]

Map:   0%|          | 0/800 [00:00<?, ? examples/s]

...

Model loaded to:  cuda


100%|██████████| 191/191 [01:24<00:00,  2.27it/s]


Model loaded to:  cuda


100%|██████████| 50/50 [00:25<00:00,  1.98it/s]


** In Domain Metrics **
Train Precision:  0.8181585021800462
Train Recall:  0.8148148148148148
Train F1:  0.8164832352188379
Test Precision:  0.7751479289940828
Test Recall:  0.7616279069767442
Test F1:  0.7683284457478006

** Cross Domain Metrics **
Train Precision:  0.6067387477998492
Train Recall:  0.5115539537841849
Train F1:  0.5550954681389464
Test Precision:  0.6033129904097646
Test Recall:  0.5199098422238918
Test F1:  0.5585149313962873


## InstructABSA - 2 - Laptops 2014

In [ ]:
task_name = 'joint_task'
model_out_path = 'kevinscaria/joint_tk-instruct-base-def-pos-neg-neut-laptops'
print('Model output path: ', model_out_path)
 
# Load the data
id_train_file_path = './Dataset/SemEval14/Train/Laptops_Train.csv'
id_test_file_path = './Dataset/SemEval14/Test/Laptops_Test.csv'
id_tr_df = pd.read_csv(id_train_file_path)
id_te_df = pd.read_csv(id_test_file_path)

ood_train_file_path = './Dataset/SemEval14/Train/Restaurants_Train.csv'
ood_test_file_path = './Dataset/SemEval14/Test/Restaurants_Test.csv'
ood_tr_df = pd.read_csv(ood_train_file_path)
ood_te_df = pd.read_csv(ood_test_file_path)

# Get the input text into the required format using Instructions
instruct_handler = InstructionsHandler()

# Set instruction_set1 for InstructABSA-1 and instruction_set2 for InstructABSA-2
instruct_handler.load_instruction_set2()

loader = DatasetLoader(id_tr_df, id_te_df, ood_tr_df, ood_te_df)
# Set bos_instruct1 for lapt14 and bos_instruct2 for rest14. For other datasets, modify the insructions.py file.
if loader.train_df_id is not None:
    loader.train_df_id = loader.create_data_in_joint_task_format(loader.train_df_id, 'term', 'polarity', 'raw_text', 'aspectTerms', instruct_handler.joint['bos_instruct1'], instruct_handler.joint['eos_instruct'])
if loader.test_df_id is not None:
    loader.test_df_id = loader.create_data_in_joint_task_format(loader.test_df_id, 'term', 'polarity', 'raw_text', 'aspectTerms', instruct_handler.joint['bos_instruct1'], instruct_handler.joint['eos_instruct'])
if loader.train_df_ood is not None:
    loader.train_df_ood = loader.create_data_in_joint_task_format(loader.train_df_ood, 'term', 'polarity', 'raw_text', 'aspectTerms', instruct_handler.joint['bos_instruct2'], instruct_handler.joint['eos_instruct'])
if loader.test_df_ood is not None:
    loader.test_df_ood = loader.create_data_in_joint_task_format(loader.test_df_ood, 'term', 'polarity', 'raw_text', 'aspectTerms', instruct_handler.joint['bos_instruct2'], instruct_handler.joint['eos_instruct'])    
    
# Model inference - Loading from Checkpoint
t5_exp = T5Generator(model_out_path)

# Tokenize Datasets
id_ds, id_tokenized_ds, ood_ds, ood_tokenized_ds = loader.set_data_for_training_semeval(t5_exp.tokenize_function_inputs)

# Get prediction labels - In Domain Training set   
id_tr_pred_labels = t5_exp.get_labels(tokenized_dataset = id_tokenized_ds, sample_set = 'train', batch_size = 16)
id_tr_labels = [i.strip() for i in id_ds['train']['labels']]

# Get prediction labels - In Domain Testing set
id_te_pred_labels = t5_exp.get_labels(tokenized_dataset = id_tokenized_ds, sample_set = 'test', batch_size = 16)
id_te_labels = [i.strip() for i in id_ds['test']['labels']]

# Get prediction labels - Cross Domain Training set   
ood_tr_pred_labels = t5_exp.get_labels(tokenized_dataset = ood_tokenized_ds, sample_set = 'train', batch_size = 16)
ood_tr_labels = [i.strip() for i in ood_ds['train']['labels']]

# Get prediction labels - Cross Domain Testing set
ood_te_pred_labels = t5_exp.get_labels(tokenized_dataset = ood_tokenized_ds, sample_set = 'test', batch_size = 16)
ood_te_labels = [i.strip() for i in ood_ds['test']['labels']]

p, r, f1, _ = t5_exp.get_metrics(id_tr_labels, id_tr_pred_labels)
print('\n** In Domain Metrics **')
print('Train Precision: ', p)
print('Train Recall: ', r)
print('Train F1: ', f1)

p, r, f1, _ = t5_exp.get_metrics(id_te_labels, id_te_pred_labels)
print('Test Precision: ', p)
print('Test Recall: ', r)
print('Test F1: ', f1)

p, r, f1, _ = t5_exp.get_metrics(ood_tr_labels, ood_tr_pred_labels)
print('\n** Cross Domain Metrics **')
print('Train Precision: ', p)
print('Train Recall: ', r)
print('Train F1: ', f1)

p, r, f1, _ = t5_exp.get_metrics(ood_te_labels, ood_te_pred_labels)
print('Test Precision: ', p)
print('Test Recall: ', r)
print('Test F1: ', f1)

Model output path:  kevinscaria/joint_tk-instruct-base-def-pos-neg-neut-laptops


Map:   0%|          | 0/3045 [00:00<?, ? examples/s]

Map:   0%|          | 0/800 [00:00<?, ? examples/s]

Map:   0%|          | 0/3041 [00:00<?, ? examples/s]

Map:   0%|          | 0/800 [00:00<?, ? examples/s]

...

Model loaded to:  cuda


100%|██████████| 191/191 [02:19<00:00,  1.37it/s]


Model loaded to:  cuda


100%|██████████| 50/50 [00:38<00:00,  1.28it/s]


** In Domain Metrics **
Train Precision:  0.8434200360545969
Train Recall:  0.8365261813537676
Train F1:  0.8399589638368813
Test Precision:  0.8140454995054401
Test Recall:  0.7974806201550387
Test F1:  0.8056779246206558

** Cross Domain Metrics **
Train Precision:  0.6774517180837722
Train Recall:  0.5726097095611618
Train F1:  0.6206341911764706
Test Precision:  0.6990116801437556
Test Recall:  0.5845229151014275
Test F1:  0.6366612111292962


## InstructABSA - 1 - Restaurants 2014

In [ ]:
task_name = 'joint_task'
model_out_path = 'kevinscaria/joint_tk-instruct-base-def-pos-restaurants'
print('Model output path: ', model_out_path)
 
# Load the data
id_train_file_path = './Dataset/SemEval14/Train/Restaurants_Train.csv'
id_test_file_path = './Dataset/SemEval14/Test/Restaurants_Test.csv'
id_tr_df = pd.read_csv(id_train_file_path)
id_te_df = pd.read_csv(id_test_file_path)

ood_train_file_path = './Dataset/SemEval14/Train/Laptops_Train.csv'
ood_test_file_path = './Dataset/SemEval14/Test/Laptops_Test.csv'
ood_tr_df = pd.read_csv(ood_train_file_path)
ood_te_df = pd.read_csv(ood_test_file_path)

# Get the input text into the required format using Instructions
instruct_handler = InstructionsHandler()

# Set instruction_set1 for InstructABSA-1 and instruction_set2 for InstructABSA-2
instruct_handler.load_instruction_set1()

loader = DatasetLoader(id_tr_df, id_te_df, ood_tr_df, ood_te_df)
# Set bos_instruct1 for lapt14 and bos_instruct2 for rest14. For other datasets, modify the insructions.py file.
if loader.train_df_id is not None:
    loader.train_df_id = loader.create_data_in_joint_task_format(loader.train_df_id, 'term', 'polarity', 'raw_text', 'aspectTerms', instruct_handler.joint['bos_instruct2'], instruct_handler.joint['eos_instruct'])
if loader.test_df_id is not None:
    loader.test_df_id = loader.create_data_in_joint_task_format(loader.test_df_id, 'term', 'polarity', 'raw_text', 'aspectTerms', instruct_handler.joint['bos_instruct2'], instruct_handler.joint['eos_instruct'])
if loader.train_df_ood is not None:
    loader.train_df_ood = loader.create_data_in_joint_task_format(loader.train_df_ood, 'term', 'polarity', 'raw_text', 'aspectTerms', instruct_handler.joint['bos_instruct1'], instruct_handler.joint['eos_instruct'])
if loader.test_df_ood is not None:
    loader.test_df_ood = loader.create_data_in_joint_task_format(loader.test_df_ood, 'term', 'polarity', 'raw_text', 'aspectTerms', instruct_handler.joint['bos_instruct1'], instruct_handler.joint['eos_instruct'])    
    
# Model inference - Loading from Checkpoint
t5_exp = T5Generator(model_out_path)

# Tokenize Datasets
id_ds, id_tokenized_ds, ood_ds, ood_tokenized_ds = loader.set_data_for_training_semeval(t5_exp.tokenize_function_inputs)

# Get prediction labels - In Domain Training set   
id_tr_pred_labels = t5_exp.get_labels(tokenized_dataset = id_tokenized_ds, sample_set = 'train', batch_size = 16)
id_tr_labels = [i.strip() for i in id_ds['train']['labels']]

# Get prediction labels - In Domain Testing set
id_te_pred_labels = t5_exp.get_labels(tokenized_dataset = id_tokenized_ds, sample_set = 'test', batch_size = 16)
id_te_labels = [i.strip() for i in id_ds['test']['labels']]

# Get prediction labels - Cross Domain Training set   
ood_tr_pred_labels = t5_exp.get_labels(tokenized_dataset = ood_tokenized_ds, sample_set = 'train', batch_size = 16)
ood_tr_labels = [i.strip() for i in ood_ds['train']['labels']]

# Get prediction labels - Cross Domain Testing set
ood_te_pred_labels = t5_exp.get_labels(tokenized_dataset = ood_tokenized_ds, sample_set = 'test', batch_size = 16)
ood_te_labels = [i.strip() for i in ood_ds['test']['labels']]

p, r, f1, _ = t5_exp.get_metrics(id_tr_labels, id_tr_pred_labels)
print('\n** In Domain Metrics **')
print('Train Precision: ', p)
print('Train Recall: ', r)
print('Train F1: ', f1)

p, r, f1, _ = t5_exp.get_metrics(id_te_labels, id_te_pred_labels)
print('Test Precision: ', p)
print('Test Recall: ', r)
print('Test F1: ', f1)

p, r, f1, _ = t5_exp.get_metrics(ood_tr_labels, ood_tr_pred_labels)
print('\n** Cross Domain Metrics **')
print('Train Precision: ', p)
print('Train Recall: ', r)
print('Train F1: ', f1)

p, r, f1, _ = t5_exp.get_metrics(ood_te_labels, ood_te_pred_labels)
print('Test Precision: ', p)
print('Test Recall: ', r)
print('Test F1: ', f1)

Model output path:  kevinscaria/joint_tk-instruct-base-def-pos-restaurants


Map:   0%|          | 0/3041 [00:00<?, ? examples/s]

Map:   0%|          | 0/800 [00:00<?, ? examples/s]

Map:   0%|          | 0/3045 [00:00<?, ? examples/s]

Map:   0%|          | 0/800 [00:00<?, ? examples/s]

...

Model loaded to:  cuda


100%|██████████| 191/191 [01:34<00:00,  2.02it/s]


Model loaded to:  cuda


100%|██████████| 50/50 [00:23<00:00,  2.09it/s]


** In Domain Metrics **
Train Precision:  0.7611370386084005
Train Recall:  0.7606529573881704
Train F1:  0.7608949210051956
Test Precision:  0.7481089258698941
Test Recall:  0.7430503380916604
Test F1:  0.7455710516396532

** Cross Domain Metrics **
Train Precision:  0.6573705179282868
Train Recall:  0.632183908045977
Train F1:  0.6445312500000001
Test Precision:  0.6526207605344296
Test Recall:  0.6153100775193798
Test F1:  0.6334164588528679


## InstructABSA - 2 - Restaurants 2014

In [ ]:
task_name = 'joint_task'
model_out_path = 'kevinscaria/joint_tk-instruct-base-def-pos-neg-neut-restaurants'
print('Model output path: ', model_out_path)
 
# Load the data
id_train_file_path = './Dataset/SemEval14/Train/Restaurants_Train.csv'
id_test_file_path = './Dataset/SemEval14/Test/Restaurants_Test.csv'
id_tr_df = pd.read_csv(id_train_file_path)
id_te_df = pd.read_csv(id_test_file_path)

ood_train_file_path = './Dataset/SemEval14/Train/Laptops_Train.csv'
ood_test_file_path = './Dataset/SemEval14/Test/Laptops_Test.csv'
ood_tr_df = pd.read_csv(ood_train_file_path)
ood_te_df = pd.read_csv(ood_test_file_path)

# Get the input text into the required format using Instructions
instruct_handler = InstructionsHandler()

# Set instruction_set1 for InstructABSA-1 and instruction_set2 for InstructABSA-2
instruct_handler.load_instruction_set2()

loader = DatasetLoader(id_tr_df, id_te_df, ood_tr_df, ood_te_df)
# Set bos_instruct1 for lapt14 and bos_instruct2 for rest14. For other datasets, modify the insructions.py file.
if loader.train_df_id is not None:
    loader.train_df_id = loader.create_data_in_joint_task_format(loader.train_df_id, 'term', 'polarity', 'raw_text', 'aspectTerms', instruct_handler.joint['bos_instruct2'], instruct_handler.joint['eos_instruct'])
if loader.test_df_id is not None:
    loader.test_df_id = loader.create_data_in_joint_task_format(loader.test_df_id, 'term', 'polarity', 'raw_text', 'aspectTerms', instruct_handler.joint['bos_instruct2'], instruct_handler.joint['eos_instruct'])
if loader.train_df_ood is not None:
    loader.train_df_ood = loader.create_data_in_joint_task_format(loader.train_df_ood, 'term', 'polarity', 'raw_text', 'aspectTerms', instruct_handler.joint['bos_instruct1'], instruct_handler.joint['eos_instruct'])
if loader.test_df_ood is not None:
    loader.test_df_ood = loader.create_data_in_joint_task_format(loader.test_df_ood, 'term', 'polarity', 'raw_text', 'aspectTerms', instruct_handler.joint['bos_instruct1'], instruct_handler.joint['eos_instruct'])    
    
# Model inference - Loading from Checkpoint
t5_exp = T5Generator(model_out_path)

# Tokenize Datasets
id_ds, id_tokenized_ds, ood_ds, ood_tokenized_ds = loader.set_data_for_training_semeval(t5_exp.tokenize_function_inputs)

# Get prediction labels - In Domain Training set   
id_tr_pred_labels = t5_exp.get_labels(tokenized_dataset = id_tokenized_ds, sample_set = 'train', batch_size = 16)
id_tr_labels = [i.strip() for i in id_ds['train']['labels']]

# Get prediction labels - In Domain Testing set
id_te_pred_labels = t5_exp.get_labels(tokenized_dataset = id_tokenized_ds, sample_set = 'test', batch_size = 16)
id_te_labels = [i.strip() for i in id_ds['test']['labels']]

# Get prediction labels - Cross Domain Training set   
ood_tr_pred_labels = t5_exp.get_labels(tokenized_dataset = ood_tokenized_ds, sample_set = 'train', batch_size = 16)
ood_tr_labels = [i.strip() for i in ood_ds['train']['labels']]

# Get prediction labels - Cross Domain Testing set
ood_te_pred_labels = t5_exp.get_labels(tokenized_dataset = ood_tokenized_ds, sample_set = 'test', batch_size = 16)
ood_te_labels = [i.strip() for i in ood_ds['test']['labels']]

p, r, f1, _ = t5_exp.get_metrics(id_tr_labels, id_tr_pred_labels)
print('\n** In Domain Metrics **')
print('Train Precision: ', p)
print('Train Recall: ', r)
print('Train F1: ', f1)

p, r, f1, _ = t5_exp.get_metrics(id_te_labels, id_te_pred_labels)
print('Test Precision: ', p)
print('Test Recall: ', r)
print('Test F1: ', f1)

p, r, f1, _ = t5_exp.get_metrics(ood_tr_labels, ood_tr_pred_labels)
print('\n** Cross Domain Metrics **')
print('Train Precision: ', p)
print('Train Recall: ', r)
print('Train F1: ', f1)

p, r, f1, _ = t5_exp.get_metrics(ood_te_labels, ood_te_pred_labels)
print('Test Precision: ', p)
print('Test Recall: ', r)
print('Test F1: ', f1)

Model output path:  kevinscaria/joint_tk-instruct-base-def-pos-neg-neut-restaurants


Map:   0%|          | 0/3041 [00:00<?, ? examples/s]

Map:   0%|          | 0/800 [00:00<?, ? examples/s]

Map:   0%|          | 0/3045 [00:00<?, ? examples/s]

Map:   0%|          | 0/800 [00:00<?, ? examples/s]

...

Model loaded to:  cuda


100%|██████████| 191/191 [02:03<00:00,  1.55it/s]


Model loaded to:  cuda


100%|██████████| 50/50 [00:31<00:00,  1.61it/s]


** In Domain Metrics **
Train Precision:  0.8242137009909521
Train Recall:  0.8111087555649777
Train F1:  0.8176087188802221
Test Precision:  0.7900302114803626
Test Recall:  0.7858752817430503
Test F1:  0.7879472693032016

** Cross Domain Metrics **
Train Precision:  0.6740157480314961
Train Recall:  0.6559386973180077
Train F1:  0.6648543689320389
Test Precision:  0.6748717948717948
Test Recall:  0.6375968992248062
Test F1:  0.6557050323866468
